In [1]:
!pip install -U pip
!pip install -U setuptools wheel

#CPU version of pytorch has smaller footprint - see installation instructions in
#pytorch documentation - https://pytorch.org/get-started/locally/
!pip install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.8 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3
    Uninstalling pip-22.3:
      Successfully uninstalled pip-22.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 38.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('full_dataset.csv')

In [4]:
df.head()

,person_id,event,time,offer_id,amount_spent,gender,age,became_member_on,income,member_duration_days,difficulty,duration,reward_per_hour,email,mobile,social,web,bogo,discount,informational
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,2017-05-09,100000.0,443,5.0,168.0,0.029762,1.0,1.0,0.0,1.0,1.0,0.0,0.0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,2017-05-09,100000.0,443,5.0,168.0,0.029762,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,132,0,19.89,F,75,2017-05-09,100000.0,443,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,2017-05-09,100000.0,443,5.0,168.0,0.029762,1.0,1.0,0.0,1.0,1.0,0.0,0.0
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,144,0,17.78,F,75,2017-05-09,100000.0,443,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df_used = df.copy()
df_used = df_used[(df_used.event == 'transaction') | (df_used.event == 'offer completed')].reset_index(drop=True)
df_used['buy_with_offer'] = df_used['event'].apply(lambda x: 1 if x == 'offer completed' else 0)

In [65]:
df_used.head()

,person_id,event,time,offer_id,amount_spent,gender,age,became_member_on,income,member_duration_days,...,duration,reward_per_hour,email,mobile,social,web,bogo,discount,informational,buy_with_offer
0,78afa995795e4d85b5d9ceeca43f5fef,transaction,132,0,19.89,F,75,2017-05-09,100000.0,443,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,2017-05-09,100000.0,443,...,168.0,0.029762,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,144,0,17.78,F,75,2017-05-09,100000.0,443,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,78afa995795e4d85b5d9ceeca43f5fef,transaction,222,0,19.67,F,75,2017-05-09,100000.0,443,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,240,0,29.72,F,75,2017-05-09,100000.0,443,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Split the data into training and test sets
train, test = train_test_split(df_used, test_size=0.2, random_state=42)

#scaler = StandardScaler()
#train = scaler.fit_transform(train)
#test = scaler.fit_transform(test)

In [67]:
from autogluon.tabular import TabularDataset, TabularPredictor
#from autogluon.core.features import StandardScaler

In [68]:
tain_data = TabularDataset(train)

In [69]:
predictor = TabularPredictor(
    label="buy_with_offer",
    eval_metric='f1',
    learner_kwargs={
        "ignored_columns": ['event', 'person_id', 'offer_id', 'became_member_on']
    }
)

predictor.fit(tain_data, time_limit=600, presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels/ag-20230415_133339/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20230415_133339/"
AutoGluon Version:  0.7.0
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #30-Ubuntu SMP Wed Nov 23 14:15:21 UTC 2022
Train Data Rows:    125120
Train Data Columns: 20
Label Column: buy_with_offer
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Usi

In [70]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,1.000000,0.129878,4.678498,0.129878,4.678498,1,True,4
1,LightGBMXT_BAG_L1,1.000000,0.233334,5.583176,0.233334,5.583176,1,True,3
2,CatBoost_BAG_L1,1.000000,0.494043,127.937608,0.494043,127.937608,1,True,7
3,WeightedEnsemble_L2,1.000000,1.004158,245.581211,0.510115,117.643602,2,True,11
4,RandomForestEntr_BAG_L1,1.000000,4.434175,4.039920,4.434175,4.039920,1,True,6
5,ExtraTreesGini_BAG_L1,1.000000,4.462540,3.495203,4.462540,3.495203,1,True,8
6,RandomForestGini_BAG_L1,1.000000,4.476395,4.000432,4.476395,4.000432,1,True,5
7,ExtraTreesEntr_BAG_L1,1.000000,4.572967,3.404350,4.572967,3.404350,1,True,9
8,NeuralNetFastAI_BAG_L1,1.000000,4.599176,777.664150,4.599176,777.664150,1,True,10
9,KNeighborsDist_BAG_L1,0.991163,2.790604,1.177820,2.790604,1.177820,1,True,2


In [ ]:
data.columns

In [22]:
data.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [72]:
# Import required libraries
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import preprocessing
import pandas as pd
import graphviz

# Load Starbucks customer data
#data = pd.read_csv('full_dataset.csv').drop(columns=['person_id', 'offer_id', 'became_member_on'])

# Encode categorical features as numerical values
le = preprocessing.LabelEncoder()
#data['gender'] = le.fit_transform(data['gender'])

# Separate features and labels
X = train.drop(['buy_with_offer'], axis=1)
y = train['buy_with_offer']

# Train a decision tree classifier
dt = DecisionTreeClassifier()
dt.fit(X, y)

# Generate a visual representation of the decision tree
dot_data = export_graphviz(dt, out_file=None, feature_names=X.columns.tolist(), class_names=['offer received', 'offer viewed', 'transaction', 'offer completed'], filled=True, rounded=True)

# Display the decision tree
#graph = graphviz.Source(dot_data)
#graph.view()

ValueError: could not convert string to float: '6747dda08bed46298c23e05b64820a3a'

In [ ]:
# Import required libraries
from sklearn.tree import DecisionTreeClassifier, export_text

# Load Starbucks customer data
data = pd.read_csv('full_dataset.csv').drop(columns=['person_id', 'offer_id', 'became_member_on'])

# Encode categorical features as numerical values
le = preprocessing.LabelEncoder()
data['gender'] = le.fit_transform(data['gender'])

# Separate features and labels
X = data.drop(['event'], axis=1)
y = data['event']

# Generate a text-based representation of the decision tree
tree_rules = export_text(dt, feature_names=X.columns.tolist())

# Print the decision tree rules
print(tree_rules)
